# 4.5 学習アルゴリズムの実装

まず、復習を兼ねてニューラルネットの学習手順を確認します。

<b><font size=4.5>ステップ１（ミニバッチ）</font></b> <br>
　訓練データの中からランダムに一部のデータを選び出す（ミニバッチ）。そのミニバッチの損失関数値を減らすことを目的とする<br><br>

<b><font size=4.5>ステップ２（勾配の算出）</font></b> <br>
　ミニバッチの損失関数を減らすために、各重みパラメータの勾配を求める。勾配は損失関数を最も減らす方向を示す<br><br>
 
<b><font size=4.5>ステップ３（パラメータの更新）</font></b> <br>
　重みパラメータを勾配の方向に微笑量だけ更新する<br><br>
 
<b><font size=4.5>ステップ４（繰り返し）</font></b> <br>
　ステップ１、ステップ２、ステップ３を繰り返す<br><br>
 
 ニューラルネットワークの学習はこの４ステップで行います。この方法は使用するデータをミニバッチで無作為に選んだデータを勾配降下法で更新しているため、<b>確率的勾配降下法(stochastic gradient descent)</b>と呼ばれます。ちなみに頭文字をとって<b><font size=4.5>ＳＧＤ</font></b>とも呼ばれます。それでは2層のニューラルネットワークを対象にＭＮＩＳＴのデータを使って学習を行います。<br><br>
 
 # 4.5.1 ２層ニューラルネットワークのクラス
 　まずは、2層ニューラルネット（隠れ層が1層のネットワーク）をひとつのクラスとして実装します。

In [1]:
###jupyterとゼロディープで使うモジュールのある場所が異なるのでこのように章のディレクトリパスをはる
###パスは皆さんの環境に合せて書き換えてください
import sys, os
sys.path.append('C:\\Users\\satoshi\\Desktop\\DeepLearning_Study\\zero_DL\\')

from common.functions import *
from common.gradient import numerical_gradient


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:入力データ, t:教師データ
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads